In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numba import njit

from twpasolver.frequency import Frequencies
from twpasolver.models import TWPA, LCLfBaseCell, StubBaseCell

freqs = np.arange(1e9, 12e9, 2e6)
N_u = 60
N_l = 6
N_sc = 1000
N = N_sc * (N_u + N_l)

# Cell capacitances, expressed in F
C_u = 18.81e-15
C_l = 7.0e-15
C_eff = (N_u * C_u + N_l * C_l) / (N_u + N_l)

# Cell inductances, expressed in H
L_u = 45.2e-12
L_l = 45.2e-12
L_eff = (N_u * L_u + N_l * L_l) / (N_u + N_l)

# Finger inductances, expressed in H
L_f_u = 1.02e-9
L_f_l = 0.335e-9
L_f_eff = (N_u * L_f_u + N_l * L_f_l) / (N_u + N_l)

# Characteristic inductance
Z_0 = np.sqrt(L_u / C_u)
Z_l = np.sqrt(L_l / C_l)
Z_eff = np.sqrt(L_eff / C_eff)

l1_u = 102e-6
l1_l = 33.5e-6
l2 = 2e-6

In [ ]:
u = LCLfBaseCell(C=C_u, L=L_u, Lf=L_f_u, Z0=Z_0, N=N_u / 2)
l = LCLfBaseCell(C=C_l, L=L_l, Lf=L_f_l, Z0=Z_l, N=N_l)
eff = LCLfBaseCell(C=C_eff, L=L_eff, Lf=L_f_eff, Z0=Z_eff, N=N_u + N_l)
twpa = TWPA(cells=[u, l, u], N=N_sc, Z0=Z_0)
twpa_not_loaded = TWPA(cells=[eff], N=N_sc, Z0=Z_eff)

# Stub model, TODO use better parameters
us = StubBaseCell(l1=l1_u, l2=l2, C=C_u, L=L_u, Lf=L_f_u, Z0=Z_0, N=N_u / 2, n_stub=2)
ls = StubBaseCell(l1=l1_l, l2=l2, C=C_l, L=L_l, Lf=L_f_l, Z0=Z_0, N=N_l, n_stub=2)
twpa_stub = TWPA(cells=[us, ls, us], N=N_sc, Z0=Z_0)

### Simple dumping and loading of configuration files

In [ ]:
twpa.dump_to_file("model_1")
twpa2 = TWPA.from_file("model_1")
print(twpa2 == twpa)

### S matrix parameters

In [ ]:
cell = twpa.get_network(freqs)
cell_u = twpa_not_loaded.get_network(freqs)
cell_stub = twpa_stub.get_network(freqs)

In [ ]:
cell.s21.plot_s_db(label="malnou")
cell_stub.s21.plot_s_db(label="stub")
plt.grid()
plt.ylabel("$|S_{21}|$ (dB)")

### Non-linear dispersion relation

In [ ]:
cell = twpa.get_network(freqs)
linear_disp = 2 * np.pi * freqs * np.sqrt(C_eff * L_eff)
plt.plot(
    freqs * 1e-9,
    -cell.s21.s_rad_unwrap.flatten() / N - linear_disp,
    label="with loaded cells",
)
plt.plot(
    freqs * 1e-9,
    -cell_u.s21.s_rad_unwrap.flatten() / N - linear_disp,
    label="no loaded cells",
)
plt.xlabel("Frequencies [GHz]")
plt.ylabel("$k^*$ [rad]")
plt.legend()
plt.grid()